# Session 1: Time Series Data

## Loading Data

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("DASH_A1.csv")

df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1184 entries, 0 to 1183
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    1184 non-null   object 
 1   Close   1159 non-null   float64
 2   High    1160 non-null   float64
 3   Low     1168 non-null   float64
 4   Open    1167 non-null   float64
 5   Volume  1158 non-null   float64
dtypes: float64(5), object(1)
memory usage: 55.6+ KB


## Date Format Change

We convert the 'Date' column to a datetime object because pandas can recognise and efficiently work with datetime objects. We set the `Date` column as the index because in time-series data like ours, operations are time-based.

In [ ]:
df["Date"] = pd.to_datetime(df["Date"], format="%d-%m-%Y")
df.set_index("Date", inplace=True)
df.head()



,Close,High,Low,Open,Volume
Date,,,,,
2022-02-24,100.419998,100.919998,85.177002,86.879997,6639000.0
2024-08-01,108.199997,112.769997,105.905998,108.620003,7965400.0
2025-02-11,193.089996,194.000000,189.500000,190.919998,6771900.0
2021-04-13,149.460007,150.360001,143.550003,146.839996,2823500.0
2024-09-17,129.880005,131.369995,126.900002,131.350006,2825500.0



## Ordering and Duplicates

First let's start with sorting the index.

In [28]:
df.index.is_monotonic_increasing

False

It 's not increasing.

In [30]:
df.sort_index(inplace=True)

df.index.is_monotonic_increasing

df

,Close,High,Low,Open,Volume
Date,,,,,
2020-12-09,189.509995,195.500000,163.800003,182.000000,25373700.0
2020-12-10,186.000000,187.695007,172.636002,179.710007,NaN
2020-12-11,175.000000,182.000000,168.250000,176.520004,4760600.0
2020-12-14,160.000000,170.000000,151.199997,169.100006,7859600.0
2020-12-15,158.889999,161.419998,153.759995,157.100006,5017000.0
...,...,...,...,...,...
2025-06-09,217.490005,219.830002,216.955002,218.029999,2710300.0
2025-06-10,214.970001,219.210007,210.927002,216.589996,3916700.0
2025-06-11,217.800003,219.529999,212.240005,214.184998,3091500.0



### Check Duplicates

In [31]:
df.duplicated().sum()


np.int64(50)

In [33]:
df.drop_duplicates(inplace=True)
df



,Close,High,Low,Open,Volume
Date,,,,,
2020-12-09,189.509995,195.500000,163.800003,182.000000,25373700.0
2020-12-10,186.000000,187.695007,172.636002,179.710007,NaN
2020-12-11,175.000000,182.000000,168.250000,176.520004,4760600.0
2020-12-14,160.000000,170.000000,151.199997,169.100006,7859600.0
2020-12-15,158.889999,161.419998,153.759995,157.100006,5017000.0
...,...,...,...,...,...
2025-06-09,217.490005,219.830002,216.955002,218.029999,2710300.0
2025-06-10,214.970001,219.210007,210.927002,216.589996,3916700.0
2025-06-11,217.800003,219.529999,212.240005,214.184998,3091500.0


In [34]:
df.duplicated().sum()

np.int64(0)

# Data Cleaning

## Not a number (NaN)

### Missing Values

In [36]:
df.isnull().sum()

Close     23
High      24
Low       15
Open      15
Volume    26
dtype: int64

In [37]:
df.isnull().sum().sum()

np.int64(103)

## Open Prices
Missing values in the Open column are filled with the Close of the day before as an approximation, ignoring overnight trading.

In [38]:
df["Open"] = df["Open"].fillna(df["Close"].shift(1))
